# 准备

In [2]:
import asyncio
import aiohttp
import time
import random
import requests
import pymongo
from lxml import etree
from parsel import Selector

client = pymongo.MongoClient(host='localhost', port=27017)
db = client['indianhospital']
all_page_col          = db['all_page']
hospital_col          = db['hospital']
hospital_detail_info  = db['hospital_detail_info']
headers = {
    'sec-ch-ua' : '"Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"',
    'sec-ch-ua-mobile'  : '?0',
    'sec-ch-ua-platform': '"Windows"',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
}

def random_proxy():
    proxy = 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()
    return proxy

def random_proxies():
    proxies = {
        'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()
    }
    return proxies
random_proxy(), random_proxies()

('http://186.5.94.202:999', {'http': 'http://147.28.155.79:3128'})

# 采集

### 1、 **`requests`** 采集测试

In [7]:
i = 0
for hospital in hospital_col.find()[0:10]:
    try:
        hospital_res = requests.get(
            url     = hospital['hospital_href'],
            headers = headers,
            proxies = random_proxies(),
        )
        i += 1
    except:
        continue
print(i)
# 采集十个医院耗时 53秒

### 2、**`aiohttp`** 测试

In [9]:
import nest_asyncio
nest_asyncio.apply()

async def request(hospital):
    hospital_name   = hospital['hospital_name']
    hospital_href   = hospital['hospital_href']
    hospital_status = hospital['hospital_status']
    if hospital_status == 1:
        pass
    else:
        async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=False)) as session:
            condition = {'hospital_href':hospital_href}
            update_hospital = hospital_col.find_one(condition)
            try:
                async with session.get(url=hospital_href, headers=headers, proxy=random_proxy(), timeout=10) as response:
                    print(response.status)
                    # html = await response.text()
                    # selector = Selector(text = html)
                    # hospital_name = selector.xpath('//div[@class="mi-bg-1"]/../h2/text()').re_first('Address of (.*)')
                    # hospital_address       = ', '.join(selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p[1]//text()').re('\s*(\w.*\w)\s*,*'))
                    # hospital_director      = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Director")]/../text()').re_first('\s*(\w.*\w)\s*')
                    # hospital_email         = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Email")]/../span/text()').re_first('\s*(\w.*\w)\s*')
                    # hospital_phone         = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Phone")]/../span/text()').re_first('\s*(\w.*\w)\s*')
                    # hospital_mobile        = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Mobile")]/../span/text()').re_first('\s*(\w.*\s)\s*')

                    # print(hospital_name)
                    # hospital_detail_info = {
                    #     'hospital_name'    : hospital_name,
                    #     'hospital_address' : hospital_address,
                    #     'hospital_director': hospital_director,
                    #     'hospital_email'   : hospital_email,
                    #     'hospital_phone'   : hospital_phone,
                    #     'hospital_mobile'  : hospital_mobile,
                    #     'hospital_href' : None,
                    #     'hospital_html' : None,
                    # }
                    # # 入库
                    # hospital_detail_info.insert_one(hospital_detail_info)
                    # # 更新状态
                    # update_hospital['hospital_status'] = 1
                    # hospital_col.update(condition, {'$set':update_hospital})
            except Exception as e:
                # 更新状态
                
                # update_hospital['hospital_status'] = 0
                # hospital_col.update(condition, {'$set':update_hospital})
                print('【ERROR】', e)

hospitals = hospital_col.find()[0:5]
tasks = [asyncio.ensure_future(request(hospital)) for hospital in hospitals]
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(*tasks))

【ERROR】 Cannot connect to host 20.87.47.204:8000 ssl:default [Connect call failed ('20.87.47.204', 8000)]
200
200
【ERROR】 
【ERROR】 


[None, None, None, None, None]

In [5]:
import requests

res = requests.get('http://localhost:5555/random')
if res.status_code == 200:
    proxy = res.text
    print(proxy)

200.25.254.193:54240


In [51]:
import requests
from scrapy import Request

response = Request(
    url = 'https://www.medindia.net/patients/hospital_search/chakraborty-hospital-port-blair-andaman-nicobar-95591-1.htm',
    meta = {'proxy':'http://'+requests.get('http://localhost:5555/random').text}
)
response

<GET https://www.medindia.net/patients/hospital_search/chakraborty-hospital-port-blair-andaman-nicobar-95591-1.htm>

In [52]:
proxy = 'http://'+requests.get('http://localhost:5555/random').text
print(proxy)

res = requests.get(proxy)

KeyboardInterrupt: 

In [ ]:
try:
    response = requests.get(proxy_url)
    if response.status_code == 200:
        proxy = response.text
        return proxy
except requests.ConnectionError:
    return False